In [5]:
same_script_path = "../samples/functions.py"
keywords_file_path = "../data/keywords.dat"

In [6]:
sample_file = open(same_script_path, "r")
sample_file_contents = sample_file.read()
sample_file.close()

In [7]:
def find_str(str_to_search, sub_str): 
    try:
        index = str_to_search.index(sub_str)
    except:
        return -1
    return index

In [8]:
def extract_functions(filepath):
    sample_file = open(filepath, "r")
    sample_file_contents = sample_file.read()
    sample_file.close()

    sample_file_lines = sample_file_contents.split("\n")

    function_names = []
    for line in sample_file_lines:
        if find_str(line, "def") != -1:
            function_parts = line.split("def ")
            func_name = function_parts[1].split("(")[0]
            function_names.append(func_name)

    return function_names

In [9]:
extract_functions(same_script_path)

['hello', 'add', 'subtract', 'multiply']

In [10]:
def hamdist(str1, str2):        
    diffs = 0
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
    return diffs

In [11]:
hamdist("prin", "some_random_string") 

4

In [12]:
import numpy as np

def levenshtein(seq1, seq2):  
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in xrange(size_x):
        matrix [x, 0] = x
    for y in xrange(size_y):
        matrix [0, y] = y

    for x in xrange(1, size_x):
        for y in xrange(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    return (matrix[size_x - 1, size_y - 1])

In [13]:
levenshtein("pront", "false")

NameError: name 'xrange' is not defined

In [14]:
!pip install pyjarowinkler

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [15]:
from pyjarowinkler import distance

def jaro_winkler(seq1, seq2):
    return distance.get_jaro_distance(seq1, seq2, winkler=True, scaling=0.1)

In [16]:
jaro_winkler("prnt", "print")

0.95

In [17]:
def get_python_keywords():
    keywords_file = open(keywords_file_path, "r")
    keywords_file_content = keywords_file.read()
    keywords_file.close()
    
    return keywords_file_content.split("\n")
    
get_python_keywords()

['and',
 'as',
 'assert',
 'async',
 'await',
 'break',
 'class',
 'continue',
 'def',
 'del',
 'elif',
 'else',
 'except',
 'exec',
 'False',
 'finally',
 'for',
 'from',
 'global',
 'if',
 'import',
 'in',
 'is',
 'lambda',
 'None',
 'nonlocal',
 'not',
 'or',
 'pass',
 'print',
 'raise',
 'return',
 'True',
 'try',
 'while',
 'with',
 'yield']

In [18]:
def closest_keyword(filepath, seq):
    keywords = extract_functions(same_script_path) + get_python_keywords()
    highest_score = -1
    closest_word = ""
    for word in keywords:
        score= jaro_winkler(seq, word)

        if score > highest_score:
            highest_score = score
            closest_word = word
    
    return closest_word

closest_keyword(same_script_path, "sbutract")

'subtract'

In [19]:
def classify_error(code_string):
    try:
        exec(code_string)
    except IndexError:
        return "IndexError" # Index in an array doesn't exist
    except TypeError:
        return "TypeError" # Attempting to mix two types that can't operate with one another
    except ZeroDivisionError: 
        return "ZeroDivisionError" # When dividing by zero
    except NameError: 
        return "NameError" #  when a name/variable does not exist

In [20]:
classify_error("1/0")

'ZeroDivisionError'

In [21]:
arr = [1]
classify_error("arr[4]")

'IndexError'

In [22]:
def fix_typos(error_script_path):
    file_ref = open(error_script_path, "r")
    file_contents = file_ref.read()
    file_ref.close()

    file_lines = file_contents.split("\n")
    new_lines = ""
    for line in file_lines:
        calls = line.split("(")
        #print(line)
        
        fixes = []
        for call in calls:
            # checking for the call not being a def
            if (find_str(call, "def") == -1) and (find_str(call, ",") == -1) \
                and (find_str(call, ")") == -1) :
                parms = call.split(" ")
                parms = list(filter(None, parms)) 

                if len(parms) > 0:
                    function_name = parms[0]
                    closest = closest_keyword(error_script_path, function_name)
                    fixes.append((function_name, closest))
        
        fixed = line
        for fix in fixes:
            fixed = line.replace(fix[0], fix[1])
        
        new_lines += fixed + "\n"
    
    return new_lines

In [23]:
error_script_path = "../samples/script_with_error.py"
fix_typos(error_script_path)

'def add(a,b):\n    return a + b\n\nprint(add(1,2))\nprint("hello")\n'

In [24]:
a = "abcd"
a.replace("a", "123")

'123bcd'

In [25]:
import linecache
import sys

def identify_exeception():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION FOUND IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))
    return (filename, lineno, line.strip(), exc_obj)


In [88]:
def error_line(code):
    n = 1
    for line in code.split("\n"):
        try:
            exec(line)
            n = n + 1
        except:
            break

    return (n, line)

In [ ]:
extrac_type(line)

In [137]:
import re

def classify_error(code_string):
    try:
        exec(code_string)
    except NameError:
        exception = identify_exeception()
        err_vars = re.findall(r'\'\w+\'', str(exception[3]))
        return ("NameError", err_vars[0].replace("'", ""))
    
    except TypeError:
        exception = identify_exeception()
        line = error_line(code_string)
        error_type = re.findall(r'(must be )(\w+)', str(exception[3]))[0][1]
        return ("TypeError", (error_type, line))
    
    except ZeroDivisionError:
        line = error_line(code_string)
        return("ZeroDivisionError", line)
    
    except IndexError:
        line = error_line(code_string)
        return("IndexError", line)  
    
    return (None, "")


In [156]:
error_script_path = "../samples/index_error.py"
def fix_errors(error_script_path):
    file_ref = open(error_script_path, "r")
    code = file_ref.read()
    file_ref.close()

    error_type, data = classify_error(code)
    if error_type == "NameError":
        replacement = closest_keyword(error_script_path, data)
        code = code.replace(data, replacement)
        print(code)

    elif error_type == "TypeError":
        correct_type, line_info = data
        line_number, line_txt = line_info
        line_vars = line_txt.split(" ")[1:]
        filtered = []
        for t in line_vars:
            if t.isalnum():
                line_txt = line_txt.replace(t, correct_type+"("+t+")")

        lines = code.split("\n")
        lines[line_number -1] = line_txt
        new_code = "\n".join(lines)
        print(new_code)

    elif error_type == "ZeroDivisionError":
        line_number, line_txt = data
        line_txt = data[1].replace("/0", "/1")
        lines = code.split("\n")
        lines[line_number -1] = line_txt
        new_code = "\n".join(lines)
        print(new_code)

    elif error_type == "IndexError":
        line_number, line_txt = data
        lines = code.split("\n")
        print(line_txt)
        lines[line_number -1] = re.sub("([\(\[]).*?([\)\]])", "\g<1>-1\g<2>", line_txt)
        new_code = "\n".join(lines)
        print(new_code)

In [157]:
error_script_path = "../samples/zero_division.py"
fix_errors(error_script_path)

x = 1
y = x/1


In [142]:
x = "a[14]"
re.sub("([\(\[]).*?([\)\]])", "\g<1>-1\g<2>", x)

'a[-1]'

In [ ]:
while (error_type != None):
    if error_type == "NameError":
        replacement = closest_keyword(error_script_path, data)
        code = code.replace(data, replacement)
    
    error_type, data = classify_error(code)
    
print(code)    